#Part One - Scrape the list of US presidents from https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States. 
using pandas and save them as a CSV.

Cassidy Jensen
Homework 7

In [1]:
import pandas as pd

In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States')

In [4]:
#tables[0]

In [5]:
tables[0].to_csv('presidents.csv', index = False)

PART TWO - Scrape the content of https://www.lemonde.fr/Links to an external site. and save it as a CSV.

We want: titles, subhead, article URL, whether it's premium or not, byline, article type, image URL.

You're going to be missing some article URLs - you should use try/except on them to avoid them being a problem, buuuut after that I would recommend printing the element to see why your approach isn't working (it's 100% possible to get all of the URLs!)
If you want to make sure the element you're getting has a specific attribute, you can do something like .select_one("a[href]"). That will only give you anchor tags (links) that have hrefs.
Instead of yes/no for the premium question, you can think of it as "put text in this column if the article is premium, don't put text in it if it is not"

In [6]:
from bs4 import BeautifulSoup
import requests

response = requests.get("https://www.lemonde.fr/Links ")
doc = BeautifulSoup(response.text)

In [7]:
items = doc.find_all(class_='article')

In [8]:
rows = []
for item in items:
    row = {}
    row['title'] = item.find(class_='article__title').text
    try:
        row['url'] = item.find(class_="article article--nav").get('href')
    except:
        row['url'] = item.get('href', None)
    try:
        row['premium'] = item.find(class_='icon__premium').text
    except:
        pass
    try:
        row['image url'] = item.find(class_='lmd-link-clickarea__link').get('href')
    except:
        row['image url'] = item.get('href', None)
    try:
        row['subhed'] = item.find(class_="article__desc").text
    except:
        pass
    try:
        row['article__type'] = item.find(class_='article__type').text
    except: 
        pass
    try:
        row['article__byline'] = item.find(class_='article__type').text
    except: 
        pass
    rows.append(row)

In [9]:
import pandas as pd
df = pd.json_normalize(rows)
df.head()
df.to_csv("lemonde.csv", index=False)

PART 3 -
Scrape the list of third party drivers license locations from https://travel-id-documents.az.gov/authorized-third-party-driver-license-locationsLinks to an external site. but include the link. Save as a CSV. Since it's more than just the text from the table, this requires actually using BeautifulSoup :(

Tips:

You'll need user-agent headers for this one
In class we always did something like item.find('h2') or the like because there was only ever one h2 we wanted. In this case there are four <td> tags that you want the text from! You'll need to use .find_all and treat them like normal lists ("give me the text from the first one," + "give me the text from the second one" etc etc)
It's okay to have columns you don't want or need! You can always remove them later

In [10]:
response = requests.get("https://travel-id-documents.az.gov/authorized-third-party-driver-license-locations")
doc = BeautifulSoup(response.text)

In [11]:
table = doc.find(class_='table')
rows = table.select('tr')

In [12]:
data = []
for row in rows[1:]:
  result = []
  if len(row.select('a')) > 0:
    result.append(row.select('a')[0]['href'])
  else:
    result.append('')
  cells = row.select('td')
  for cell in cells:
      result.append(cell.text.strip())
  data.append(result)

In [16]:
df.head()

,link,company,address,telephone,hours
0,http://az-mvd.com/,1 Stop Title & Registration Services,"940 N. Alma School Rd., #104\n\t\t\tChandler, ...",480.821.3288,Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 9:00 a.m.-4...
1,http://az-mvd.com/,1 Stop Title & Registration Services,"5036 W. Cactus Rd., Ste. 2\n\t\t\tGlendale, AZ...",602.264.2400,Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 8:30 a.m.-4...
2,,Academy of Driving Motor Vehicle Center,"4733 E. Broadway Blvd.\n\t\t\tTucson, AZ 85711",520.750.7572,Mon.-Fri. 9 a.m.-5 p.m. and Sat. 9 a.m.-3 p.m.
3,,Arizona Auto License,"1337 W. Prince Rd\n\t\t\tTucson, AZ 85705",520.696.2023,Driver License Hours: \n\t\t\tMon.-Fri. 9 a.m....
4,,Arizona Auto License Service LLC,"1457 N. Eliseo C Felix Jr.\n\t\t\tWay, Ste. 10...",623.925.5455 or Fax 623.925.5879,Mon.-Fri. 8 a.m.-5 p.m.


In [13]:
df= pd.DataFrame(data)
df.columns = ['link', 'company', 'address', 'telephone', 'hours']
df.to_csv("driverslicenselocations.csv", index=False)

In [14]:
df.to_csv("driverslicenselocations.csv", index=False)

Part Four

Visit https://www.tnwb.uscourts.gov/Search/Search.aspxLinks to an external site. and search for "CAR." Scrape the results into a CSV, with four columns: the URL to the case, the name of the case, the category (e.g. "Judge's Opinions), the additional details (terms match/size/pdf URL).

Bonuses, if you want to get fancy:

Split up the additional details into multiple columns
Download all of the PDFs of the cases
Tips:

There are only 10 results, and so many pages! ...maybe there's a secret way to get them all on one page?
Check the class you're using and see if it matches the number of results (it probably doesn't!). Inspect the page to find out why. You have two options: use something like we did in class with item.select("h1, h2") – but slightly different, since we're talking about classes – or have two separate loops.
.split is often a convenient way to separate semi-structured text
Downloading PDFs in Python probably does not involve wget (unless you really want to)
Submit on Gracescope as either multiple notebooks or a single notebook.

In [120]:
response = requests.get("https://www.tnwb.uscourts.gov/Search/search.aspx?zoom_query=car&zoom_page=1&zoom_per_page=200&zoom_and=1&zoom_sort=0&zoom_xml=0")
doc = BeautifulSoup(response.text)
import os

In [121]:
items = doc.select('div.result_block, div.result_altblock')

In [122]:
len(items)

132

In [123]:
item.find(class_="result_title").get('href')

In [124]:
print(item.find(class_="result_title").get('href'))

None


In [125]:
rows = []
pdf_dir = "pdfs"
os.makedirs(pdf_dir, exist_ok=True)  

In [145]:
for item in items:
    row = {}
    row['name'] = item.find(class_="result_title").text
    try:
         row['url'] = item.find('a').get('href')
         if row['url'] and row['url'].endswith('.pdf'):
            pdf = requests.get(row['url'])
            with open(row['name'].replace(' ', '_') + '.pdf', 'wb') as f:
                f.write(pdf.content)
    except:
        row['url'] = item.get('href', None)
    row['category'] = item.find(class_='category').text
    row['details'] = item.find(class_='infoline').text
    rows.append(row)

In [146]:
df = pd.json_normalize(rows)
df.head()
df.to_csv("CARcourtresults.csv", index=False)

In [147]:
df_split = df['details'].str.split(expand=True)

In [148]:
df.head(10)

,name,url,category,details
0,1. JDL: 04-24318 Jacquelline D. Black [Judges'...,https://www.tnwb.uscourts.gov/Opinions/jdl/pdf...,[Judges' Opinions],Terms matched: 1 - 102k - URL: https://ww...
1,2. WHB: 95-26401 Mary Lucy Cooper [Judges' Opi...,https://www.tnwb.uscourts.gov/Opinions/whb/pdf...,[Judges' Opinions],Terms matched: 1 - 27k - URL: https://www...
2,3. GHB: 97-12368 Billy G. Woffard [Judges' Opi...,https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,[Judges' Opinions],Terms matched: 1 - 71k - URL: https://www...
3,4. JDL: 97-30580 Mary Chrlis Hurst [Judges' Op...,https://www.tnwb.uscourts.gov/Opinions/jdl/pdf...,[Judges' Opinions],Terms matched: 1 - 32k - URL: https://www...
4,5. MRH: 20-20967 Jacob Braxton Herring 20-0009...,https://www.tnwb.uscourts.gov/Opinions/mrh/pdf...,[Judges' Opinions],Terms matched: 1 - 303k - URL: https://ww...
5,6. GHB: 95-11365 Melissa L. Bryan [Judges' Opi...,https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,[Judges' Opinions],Terms matched: 1 - 52k - URL: https://www...
6,7. JDL: 09-20339 Diane M. Miller [Judges' Opin...,https://www.tnwb.uscourts.gov/Opinions/jdl/pdf...,[Judges' Opinions],Terms matched: 1 - 92k - URL: https://www...
7,8. GHB: 00-12340 Wanda K. Autry [Judges' Opini...,https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,[Judges' Opinions],Terms matched: 1 - 297k - URL: https://ww...
8,"9. GHB: 02-31651 Neil Bond Stewart, Jr. and Ti...",https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,[Judges' Opinions],Terms matched: 1 - 291k - URL: https://ww...
9,10. GHB: 96-12039 Randy and Janice Willson [Ju...,https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,[Judges' Opinions],Terms matched: 1 - 201k - URL: https://ww...
